In [1]:
import joblib
import mediapipe as mp
from keras.models import load_model
import cv2
import numpy as np


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

def extract_hand_landmarks(image):
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Draw bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Indicate left or right hand
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None
feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size=(128,128)
#def real_time_prediction(n):
def real_time_prediction(model, feature_extractor):
       # Initialize webcam
    cap = cv2.VideoCapture(0)

    while True:
    # Capture frame-by-frame
        ret, frame = cap.read()
        scale_factor = 1.5  # Adjust this value as needed
        new_width = int(frame.shape[1] * scale_factor)
        new_height = int(frame.shape[0] * scale_factor)
        resized_frame = cv2.resize(frame, (new_width, new_height))

    # Check if frame captured successfully
        if not ret:
            break

    # Preprocess the frame
        processed_img = preprocess_image(resized_frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
        
        # Extract features from CNN
            features = feature_extractor.predict(processed_img_reshaped)
        
        # Predict using the Decision Tree classifier
            prediction = dt_classifier.predict(features)
            
            # Convert integer label back to string label
            predicted_gesture = unique_labels[prediction[0]]
        
        # Display the result on the frame
        # Instead of a fixed position, compute the position dynamically
            text_position = (int(resized_frame.shape[1] * 0.05), int(resized_frame.shape[0] * 0.05))

            cv2.putText(resized_frame, predicted_gesture, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)

    # Display the frame
        cv2.imshow('Gesture Recognition', resized_frame)

    # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture
    cap.release()
    cv2.destroyAllWindows()
#real_time_prediction(5)
real_time_prediction(dt_classifier, feature_extractor)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━

In [6]:
from flask import Flask, render_template, Response, jsonify
import os
app = Flask(__name__)

# Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

'''
def extract_hand_landmarks(image):
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    return None'''
last_landmarks = None
last_bounding_box = None
last_hand_label = None
def extract_hand_landmarks(image):
    global last_landmarks, last_bounding_box, last_hand_label
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            # Draw landmarks
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Update bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Update hand label
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Store current landmarks, bounding box and label for future frames
            last_landmarks = landmarks
            last_bounding_box = (min_x, min_y, max_x, max_y)
            last_hand_label = hand_label
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    else:
        # If no hand is detected, use the last detected landmarks, bounding box, and label
        if last_landmarks and last_bounding_box:
            mp_drawing.draw_landmarks(image, last_landmarks, mp_hands.HAND_CONNECTIONS)
            min_x, min_y, max_x, max_y = last_bounding_box
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            if last_hand_label:
                cv2.putText(image, last_hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None

feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size = (128, 128)
path = "F:\Mini-Project\HandGesture-based-Food-Ordering-System\dataset_mediapipe"
def generate():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
       # resized_frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)))
        processed_img = preprocess_image(frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
            features = feature_extractor.predict(processed_img_reshaped)
            prediction = dt_classifier.predict(features)
            predicted_gesture = unique_labels[prediction[0]]
            final_output = os.path.basename(predicted_gesture)
            text_position = (int(frame.shape[1] * 0.05), int(frame.shape[0] * 0.05))
            cv2.putText(frame, final_output, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)

        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')
@app.route('/get_gesture')
def get_gesture():
    # Your gesture recognition logic here
    recognized_gesture = final_output  # Replace with actual recognized gesture
    return jsonify(gesture=recognized_gesture)

@app.route('/video_feed')
def video_feed():
    return Response(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index2.html')

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:23] "GET /static/icon1.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:23] "GET /static/icon2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/js/your_script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/background.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/menu2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/icon3.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/icon4.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:24] "GET /static/icon5.png HTTP/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:11:33] "GET /socket.io/?EIO=4&transport=polling&t=PHvtAya HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:13:32] "GET /socket.io/?EIO=4&transport=polling&t=PHvtdve HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:13:38] "GET /socket.io/?EIO=4&transport=polling&t=PHvtfNX HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 20:13:44] "GET /socket.io/?EIO=4&transport=polling&t=PHvtgrA HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

# Interface

In [4]:
import joblib
import pickle

In [5]:
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [6]:
from flask_socketio import SocketIO, emit

In [7]:
import os

In [8]:
from flask import Flask, render_template, Response, jsonify

In [14]:

app = Flask(__name__)
socketio = SocketIO(app)
# Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

last_landmarks = None
last_bounding_box = None
last_hand_label = None
def extract_hand_landmarks(image):
    global last_landmarks, last_bounding_box, last_hand_label
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks:
        for index, landmarks in enumerate(results.multi_hand_landmarks):
            # Draw landmarks
            mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Update bounding box
            lmx = [landmark.x for landmark in landmarks.landmark]
            lmy = [landmark.y for landmark in landmarks.landmark]
            min_x, max_x, min_y, max_y = int(min(lmx)*image.shape[1]), int(max(lmx)*image.shape[1]), int(min(lmy)*image.shape[0]), int(max(lmy)*image.shape[0])
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            
            # Update hand label
            hand_label = results.multi_handedness[index].classification[0].label
            hand_label = "Right" if hand_label == "Left" else "Left"
            cv2.putText(image, hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            
            # Store current landmarks, bounding box and label for future frames
            last_landmarks = landmarks
            last_bounding_box = (min_x, min_y, max_x, max_y)
            last_hand_label = hand_label
        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks_array = [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
        return landmarks_array
    else:
        # If no hand is detected, use the last detected landmarks, bounding box, and label
        if last_landmarks and last_bounding_box:
            mp_drawing.draw_landmarks(image, last_landmarks, mp_hands.HAND_CONNECTIONS)
            min_x, min_y, max_x, max_y = last_bounding_box
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)
            if last_hand_label:
                cv2.putText(image, last_hand_label, (min_x, min_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return None

def landmarks_to_heatmap(landmarks, img_size=(128, 128), blob_size=5):
    heatmap = np.zeros(img_size)
    for landmark in landmarks:
        x, y = int(landmark[0] * img_size[0]), int(landmark[1] * img_size[1])
        cv2.circle(heatmap, (x, y), blob_size, (255, 255, 255), -1)
    return heatmap

def preprocess_image(img, img_size=(128, 128)):
    landmarks = extract_hand_landmarks(img)
    if landmarks:
        heatmap = landmarks_to_heatmap(landmarks, img_size)
        return heatmap / 255.0
    return None

feature_extractor = load_model("cnn_model_3_mediapipe_90.h5")
dt_classifier = joblib.load('dt_classifier_3_mediapipe_90.pkl')
unique_labels = joblib.load('unique_labels_3_mediapipe_90.pkl')
img_size = (128, 128)
path = "F:\Mini-Project\HandGesture-based-Food-Ordering-System\dataset_mediapipe"
#def generate():

@socketio.on('start_gesture_recognition')
def get_gesture():
    # Your gesture recognition logic here
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
       # resized_frame = cv2.resize(frame, (int(frame.shape[1]*1.5), int(frame.shape[0]*1.5)))
        processed_img = preprocess_image(frame, img_size)
    
        if processed_img is not None:
            processed_img_reshaped = processed_img.reshape((-1, img_size[0], img_size[1], 1))
            features = feature_extractor.predict(processed_img_reshaped)
            prediction = dt_classifier.predict(features)
            predicted_gesture = unique_labels[prediction[0]]
            final_output = os.path.basename(predicted_gesture)
            text_position = (int(frame.shape[1] * 0.05), int(frame.shape[0] * 0.05))
            cv2.putText(frame, final_output, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
            socketio.emit('gesture_update', final_output)
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')

        #emit('gesture_update', (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n').decode('utf-8'))
       # socketio.emit('frame_update', jpeg.tobytes().decode('base64'))
        


@app.route('/video_feed')
def video_feed():
    return Response(get_gesture(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('index2.html')

if __name__ == "__main__":
    socketio.run(app, allow_unsafe_werkzeug=True)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/icon1.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/menu2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/background.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/js/your_script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/icon2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/icon3.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/icon4.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:41:22] "GET /static/icon5.png HTTP/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:44:22] "GET /static/quantity1.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:44:22] "GET /static/quantity2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:44:22] "GET /static/quantity3.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:44:22] "GET /static/quantity4.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 22:44:22] "GET /static/quantity5.png HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


...

INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:53] "GET /static/quantity1.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:53] "GET /static/quantity2.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:53] "GET /static/quantity3.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:53] "GET /static/quantity4.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:53] "GET /static/quantity5.png HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:10:58] "GET /static/ordered.png HTTP/1.1" 304 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 23:11:56] "GET /socket.io/?EIO=4&transport=websocket&sid=nVcqP3xzhlxtYP-mAABO HTTP/1.1" 200 -
